In [15]:
import cv2
import numpy as np

video = "a.mp4" # Carregar o caminho do video

cap = cv2.VideoCapture(video) # Abrir o video

ret, prev_frame = cap.read() # Ler o primeiro frame do video

prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) # Converter o frame para escala de cinza

# Loop para processar o video frame a frame
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    #Converter o quadro atual para a escala de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calcular o fluxo optico denso
    flow = cv2.calcOpticalFlowFarneback(prev_frame, gray, None, 0.5, 3, 20, 3, 5, 1.2, 0)
    
    # Desenhar o fluxo optico denso no quadro
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv = np.zeros_like(frame)
    hsv[..., 1] = 255
    hsv[..., 0] = angle * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Mostrar o quadro com o fluxo optico denso
    cv2.imshow('Optical Flow', flow_rgb)
    cv2.imshow('Frame', frame)

    # Atualizar o frame anterior
    prev_frame = gray
    
    # Sair do loop se a tecla 'q' for pressionada
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [30]:
import cv2
import numpy as np
import pygame  # Importando o pygame para som

# Inicializar o pygame
pygame.mixer.init()

# Caminho para o vídeo
video = "a.mp4"

# Caminho para o arquivo de alarme
alarm_sound = "alarme1.mp3"  # Substitua pelo caminho do seu arquivo de som

# Carregar o som do alarme
alarm = pygame.mixer.Sound(alarm_sound)

# Abrir o vídeo
cap = cv2.VideoCapture(video)

# Ler o primeiro quadro
ret, prev_frame = cap.read()

# Converter o primeiro quadro para escala de cinza
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Definir um limite de movimento para detectar mudanças significativas
movement_threshold = 5.0  # Você pode ajustar esse valor conforme necessário

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Converter o quadro atual para escala de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calcular o fluxo ótico denso entre o quadro anterior e o atual
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Converter o fluxo ótico para magnitude e ângulo
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Normalizar a magnitude para visualização
    magnitude_normalized = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    # Criar uma máscara para detectar movimento
    movement_mask = magnitude > movement_threshold

    # Verificar se há movimento significativo
    if np.any(movement_mask):
        # Tocar o som de alarme
        alarm.play()  # Tocar o som

    # Desenhar círculos nas áreas com movimento significativo
    for y in range(0, frame.shape[0], 10):  # Passos maiores para acelerar o processo
        for x in range(0, frame.shape[1], 10):
            if movement_mask[y, x]:
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)  # Desenha um círculo verde

    # Exibir o quadro com as áreas de movimento
    cv2.imshow("Monitoramento de Movimento", frame)

    # Atualizar o quadro anterior
    prev_gray = gray

    # Fechar o vídeo quando a tecla 'q' for pressionada
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Libera os recursos e fecha as janelas
cap.release()
cv2.destroyAllWindows()
